# Our modeling approach

- We fully characterize the existence of a 1-round protocol by a Mixed Integer Linear Program.


In [2]:
from sage.all import *
import itertools
import random
from sage.sat.solvers.satsolver import SAT
from sage.sat.solvers.cryptominisat import CryptoMiniSat
from sage.misc.temporary_file import atomic_write
import copy
import time

In [3]:
solver = SAT(solver="LP")
solver.add_clause((-1,2))
solver.add_clause((1,3))
solution = solver()
print ' solution =',solution

 solution = [None, False, False, True]


In [ ]:

# TODO: check correctness etc
def get_at(tup, a, l):
    # indexes from the left side of tup = (t0,0, t0,1, t1,0, t1,1, t2,0, t2,1)
    return tuple(tup[2*i + a[i]] for i in range(l))
 
def tuples_fixed_proj(l, proj_ind, proj_val):
#    h = randint(1,4000)
#    if (h == 5):
#        print '==================== DEBUGGING ========== a,g = ', proj_ind, proj_val
    tuples = list()
    x = [0 for i in range(2*l)]
    for i in range(l):
        x[i*2 + proj_ind[i]] = proj_val[i]
    for g in itertools.product(range(2), repeat = l):
        for i in range(l):
            x[i*2 + 1 - proj_ind[i]] = g[i]
#        if (h == 5):
#            print 'modified x to ', x
        tuples.append(tuple(x[:]))
    return tuples    


def find_solution(l, flag, client_bound = 1):
    # Generate a mixed integer program.
    # variables: 
    # 0. p0^i_{a,g,b} - 0/1 variable, 1 iff. p^i_{a,g,b} can be possibly non-zero.
    # 1. p^i_v - 0/1 variable, meaning v is (possibly) chosen towards the set. 
    # May as well not be chosen, but those assigned 0 must NOT be chosen.
    # 2. p^i_a,g,b - The probability that on input i, a is picked and b is the output.
    # 3. q^i_v - The probability that on input i, v is sent to the orcale by the server.
    # 4. q^i_a,g,b Auxiliary for privacy constraints.
    
    solver = MixedIntegerLinearProgram(solver='ppl')
    zeros = tuple((0 for i in range(l)))
    
    if (flag == 'client'):
        # Type 0
        p0 = solver.new_variable(integer = True, nonnegative=True)
        # Type 2
        q1 = solver.new_variable(integer = False, nonnegative=True)
        
        # client's probability distribution
        for i in range(3): 
            solver.add_constraint(sum(q1[(i, a, zeros, b)] for a in itertools.product(range(2), repeat = l) for b in range(2)) == 1)

        # client's basic constraints
        for i in range(3):                              
            for a in itertools.product(range(2), repeat = l):
                for g in itertools.product(range(2), repeat = l):
                    for b in range(2):
                        solver.add_constraint(p0[(i,a,g,b)] <= 1)
                        solver.add_constraint(p0[(i,a,g,b)] - q1[(i,a,g,b)] >= 0)

        # Enforce the existence of p^i_a's                      
        for i in range(3):
            for a in itertools.product(range(2), repeat = l):
                for g in itertools.product(range(2), repeat = l):              
                    solver.add_constraint(q1[(i,a,zeros,0)] + q1[(i,a,zeros,1)] - q1[(i,a,g,0)] -  q1[(i,a,g,1)] == 0)     
                
        # correctness constraints
        for t in itertools.product(range(2), repeat = 2*l):
            solver.add_constraint( sum(q1[(i, a, get_at(t,a,l) ,1)] for i in range(3)) == 1)
         
        
        # hack based on an observation from honest correctness + security against malicious clients
        # again, this is just a relaxation
        for i in range(3):                              
            for a in itertools.product(range(2), repeat = l):
                for j in range(3):
                    if (j > i):
                        solver.add_constraint(sum(q1[(i, a, zeros, b)] for b in range(2)) + sum(q1[(j, a, zeros, b)] for b in range(2)) <= 1)
                        
                        
        solver.set_objective(sum(-p0[(i, a, g, b)] for i in range(3)     
                             for a in itertools.product(range(2), repeat = l) 
                             for g in itertools.product(range(2), repeat = l) 
                             for b in range(2)))

        # another hack from correctness.
        eps = 0.00001
        
        for i in range(3):                              
            for b in range(2):
                solver.add_constraint(sum(q1[(i, a, g, b)] for a in itertools.product(range(2), repeat = l) 
                                      for g in itertools.product(range(2), repeat = l)) >= eps)
          
        
    elif (flag == 'server'):
        
        # Type 1
        p1 = solver.new_variable(integer = True, nonnegative=True)
        # Type 3
        p2 = solver.new_variable(integer = False, nonnegative=True)

        # Type 4
        aux = solver.new_variable(integer = False, nonnegative=True)
    
    
        # server basic constraints - Make sure these are 0-1. Also make sure the probabilities of vectors not in the support p1 is 0 
        for i in range(3):                              
            for t in itertools.product(range(2), repeat = 2*l):
                solver.add_constraint(p1[(i,t)] <= 1)
                solver.add_constraint(p1[(i,t)] - p2[(i,t)] >= 0)                  
            # server distribution constraint  
            solver.add_constraint(sum(p2[(i,t)] for t in itertools.product(range(2), repeat = 2*l)) == 1) 

        # privacy constraints
        for a in itertools.product(range(2), repeat = l):
            # validity of auxiliary simultion vectors
            for i in range(3):
                solver.add_constraint(sum(aux[(i,a,g,0)] for g in itertools.product(range(2), repeat = l)) 
                                    - sum(aux[(i,a,g,1)] for g in itertools.product(range(2), repeat = l)) == 0)

            solver.add_constraint(sum( aux[(i,a,g,0)] for i in range(3) for g in itertools.product(range(2), repeat = l)) == 1)
        
        # connection to the sender's distributions
        for a in itertools.product(range(2), repeat = l):
            for i in range(3):
                for g in itertools.product(range(2), repeat = l):
                    all = tuples_fixed_proj(l, a, g)
                    solver.add_constraint(sum(p2[(i,t)] for t in all) 
                                    - sum(aux[(j, a, g, int(j == i))] for j in range(3)) == 0)
       
        # so far, yields a very trivial solution. We add simple correctness constraints (independent of any concrete client's input):
        # we know that the same t can not appear for i1 != i2. Also, may further assume wlog. that all-0 is in support of y1
        # We use p2 instead of p2 here, which would not be a linear program.
        for i in range(3):
            for t in itertools.product(range(2), repeat = 2*l):
                for j in range(3):
                    if j<i:
                        solver.add_constraint(p2[(i,t)] + p2[(j,t)] <= 1)
            
        
        # We know that the same t can not appear for i1 != i2. Also, may further assume wlog. that all-0 is in support of y1
        # This is a relaxation of what we want
        for j in range(1,3):
            for g in itertools.product(range(2), repeat = l):
                t_s = tuples_fixed_proj(l, zeros, g)
                solver.add_constraint(sum(p2[(j, t)] for t in t_s) + sum(p2[(0, t)] for t in t_s) <= 1)
        
        solver.set_objective(sum(-p1[(i,t)] for i in range(3)                              
                                 for t in itertools.product(range(2), repeat = 2*l))) 
    
    elif (flag == 'corr'): #not fully supported for now
        
        print 'testing correctness'
        # honest correctness constraints - highly specific to the equality function
        # Type 0
        p0 = solver.new_variable(integer = True, nonnegative=True)
        # Type 2
        q1 = solver.new_variable(integer = False, nonnegative=True)
        # Type 1
        p1 = solver.new_variable(integer = True, nonnegative=True)

        for i in range(3):
            solver.add_constraint(sum(p1[(i,t)] for t in itertools.product(range(2), repeat = 2*l)) >= 1)
            
        for i in range(3):
            for g in itertools.product(range(2), repeat = l):
                solver.add_constraint(sum(p0[(i, a, g, b)] for a in itertools.product(range(2), repeat = l) 
                                          for b in range(2)) >= 1)
            
        for i in range(3):
            for a in itertools.product(range(2), repeat = l):
                for g in itertools.product(range(2), repeat = l):
                    for j in range(3):
                        t_s = tuples_fixed_proj(l, a, g)
                        forbidden_b = 1
                        if (i == j):
                            forbidden_b = 0
                        for t in t_s:
                            solver.add_constraint(p1[(j, t)] + p0[(i, a, g, forbidden_b)] <= 1)   

    
        solver.add_constraint(sum(p0[(i, a, g, b)] for i in range(3)     
                             for a in itertools.product(range(2), repeat = l) 
                             for g in itertools.product(range(2), repeat = l) 
                             for b in range(2)) >= client_bound) 

         
        solver.set_objective(sum(p0[(i, a, g, b)] for i in range(3)     
                             for a in itertools.product(range(2), repeat = l) 
                             for g in itertools.product(range(2), repeat = l) 
                             for b in range(2)) + 
                             sum(p1[(i,t)] for i in range(3) for t in itertools.product(range(2), repeat = 2*l)))    
            
    res = solver.solve()
    
    print 'result of solution process is ',res  
    if (flag == 'server' or flag == 'corr'):
        print 'p1 values  = \n'
        for i in range(3):
            for t in itertools.product(range(2), repeat = 2*l):
                if (solver.get_values(p1[(i,t)]) > 0):
                    print (i,t),': ',solver.get_values(p1[(i,t)])
    
    if (flag == 'server'):
        print 'p2 values  = \n'
        for i in range(3):
            for t in itertools.product(range(2), repeat = 2*l):
                if solver.get_values(p2[(i,t)]) > 0:
                    print (i,t),': ',solver.get_values(p2[(i,t)])

    if (flag == 'client'):            
        print 'q1 values  = \n'
        for i in range(3):
            for g in itertools.product(range(2), repeat = l):
                for a in itertools.product(range(2), repeat = l):
                    for b in range(2):
                        if solver.get_values(q1[(i, a, g, b)]) > 0:
                            print (i, a, g, b),': ',solver.get_values(q1[(i, a, g, b)]),'...',solver.get_values(p0[(i, a, g, b)])
                            
    if (flag == 'corr'):            
        print 'q1 values  = \n'
        for i in range(3):
            for g in itertools.product(range(2), repeat = l):
                for a in itertools.product(range(2), repeat = l):
                    for b in range(2):
                        if solver.get_values(q1[(i, a, g, b)]) > 0:
                            print (i, a, g, b),': ',solver.get_values(p0[(i, a, g, b)])
        
        
    if (flag == 'server+'):                    
        print 'aux values  = \n'
        for i in range(3):
            for a in itertools.product(range(2), repeat = l):
                for b in range(2):
                    for g in itertools.product(range(2), repeat = l):
                        print (i, a, g, b),': ',solver.get_values(aux[(i, a, g, b)])


                        
# main result    
l=2
print 'finding a solution for l=',l
find_solution(l, 'client', 12)
